In [1]:
import pyvisa,time
from decode_scope_data import LecroyScopeData
import numpy as np
from scipy.integrate import simpson

In [3]:
#Modify the following global variables according to the model
DSO_RSC = "TCPIP0::10.195.48.73::inst0::INSTR"

rm = pyvisa.ResourceManager('@py')
dso = rm.open_resource(DSO_RSC)
dso.timeout = 30000 #default value is 2000(2s)
dso.chunk_size = 20*1024*1024 #default value is 20*1024(20k bytes)

print(dso.query("*IDN?").strip())

Lecory,T3DSO2502A,T0101C21490989,5.0.1.3.9R6


In [ ]:
dso.write("TRIG:MODE SINGLE")
i = 0

with open("./SiPMT_data2.txt","w", encoding="utf-8") as f:
    while True:
        while dso.query("TRIGGER:STAT?").strip() != 'Stop':
            time.sleep(0.25)
            pass
        
        dso.write("WAV:SOUR C4")
        dso.write("WAV:PREamble?")
        recv_preamble = dso.read_raw()

        dso.write("WAV:DATA?")
        recv_data = dso.read_raw()

        waveform = LecroyScopeData(recv_preamble,recv_data)
        voltage = waveform.y
        t = waveform.x
        
        with open("./SiPM_wav.txt", "w", encoding="utf-8") as ff:
            ff.write(t, ',', voltage)

        m = max(voltage)
        index_max = np.where(m == voltage)[0][0]
        ground = np.average(voltage[0:index_max-10])
        y = voltage-ground
        integral = simpson(y,t)

        out = str(m) + ',' + str(integral)
        f.write(out + '\n')
        i += 1
        print('Counts:', i)

        dso.write("TRIG:MODE SINGLE")

Counts: 1
Counts: 2
Counts: 3
Counts: 4
Counts: 5
Counts: 6
Counts: 7
Counts: 8
Counts: 9
Counts: 10
Counts: 11
Counts: 12
Counts: 13
Counts: 14
Counts: 15
Counts: 16
Counts: 17
Counts: 18
Counts: 19
Counts: 20
Counts: 21
Counts: 22
Counts: 23
Counts: 24
Counts: 25
Counts: 26
Counts: 27
Counts: 28
Counts: 29
Counts: 30
Counts: 31
Counts: 32
Counts: 33
Counts: 34
Counts: 35
Counts: 36
Counts: 37
Counts: 38
Counts: 39
Counts: 40
Counts: 41
Counts: 42
Counts: 43
Counts: 44
Counts: 45
Counts: 46
Counts: 47
Counts: 48
Counts: 49
Counts: 50
Counts: 51
Counts: 52
Counts: 53
Counts: 54
Counts: 55
Counts: 56
Counts: 57
Counts: 58
Counts: 59
Counts: 60
Counts: 61
Counts: 62
Counts: 63
Counts: 64
Counts: 65
Counts: 66
Counts: 67
Counts: 68
Counts: 69
Counts: 70
Counts: 71
Counts: 72
Counts: 73
Counts: 74
Counts: 75
Counts: 76
Counts: 77
Counts: 78
Counts: 79
Counts: 80
Counts: 81
Counts: 82
Counts: 83
Counts: 84
Counts: 85
Counts: 86
Counts: 87
Counts: 88
Counts: 89
Counts: 90
Counts: 91
Counts: 

/home/josecds/Desktop/Projects/venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:571: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/home/josecds/Desktop/Projects/venv/lib/python3.12/site-packages/numpy/_core/_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Counts: 100
Counts: 101
Counts: 102
Counts: 103
Counts: 104
Counts: 105
Counts: 106
Counts: 107
Counts: 108
Counts: 109
Counts: 110
Counts: 111
Counts: 112
Counts: 113
Counts: 114
Counts: 115
Counts: 116
Counts: 117
Counts: 118
Counts: 119
Counts: 120
Counts: 121
Counts: 122
Counts: 123
Counts: 124
Counts: 125
Counts: 126
Counts: 127
Counts: 128
Counts: 129
Counts: 130
Counts: 131
Counts: 132
Counts: 133
Counts: 134
Counts: 135
Counts: 136
Counts: 137
Counts: 138
Counts: 139
Counts: 140
Counts: 141
Counts: 142
Counts: 143
Counts: 144
Counts: 145
Counts: 146
Counts: 147
Counts: 148
Counts: 149
Counts: 150
Counts: 151
Counts: 152
Counts: 153
Counts: 154
Counts: 155
Counts: 156
Counts: 157
Counts: 158
Counts: 159
Counts: 160
Counts: 161
Counts: 162
Counts: 163
Counts: 164
Counts: 165
Counts: 166
Counts: 167
Counts: 168
Counts: 169
Counts: 170
Counts: 171
Counts: 172
Counts: 173
Counts: 174
Counts: 175
Counts: 176
Counts: 177
Counts: 178
Counts: 179
Counts: 180
Counts: 181
Counts: 182
Coun

KeyboardInterrupt: 